#Na początku instaluję i importuję potrzebne pakiety

In [ ]:
!pip install transformers
!pip install numpy
!pip install pandas
!pip install scipy

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

     |████████████████████████████████| 2.1MB 32.2MB/s 
     |████████████████████████████████| 901kB 36.0MB/s 
     |████████████████████████████████| 3.3MB 35.2MB/s 


#Teraz wybieram interesujący mnie model.
####Dostępne taski: 
emoji, emotion, hate, irony, offensive, sentiment

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [ ]:

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Tworzę też funkcje ułatwiające tworzenie predykcji i jej wyświetlanie

In [ ]:
def predict_text(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores

def print_scores(scores):
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")


Kilka przykładowych recenzji, kilka negatywnych, kilka pozytywnych i jedna neutralna. Modelowi poszło całkiem nieźle :)

In [ ]:
# show predicted sentiment for given text
reviews = [
           'I hate this company!',
           'This is the last company I would want to work for',
           "This is not the best company in the world",

           'I kinda like this company',
           "I am constantly learning something at this company",
           "I'm glad I came here",

           'The company has been on the market for 10 years'
]

for review in reviews:
  print(f'Sentiment of the review: "{review}"')
  score = predict_text(review)
  print_scores(score)
  print('##########################')

Sentiment of the review: "I hate this company!"
1) negative 0.9801
2) neutral 0.0159
3) positive 0.004
##########################
Sentiment of the review: "This is the last company I would want to work for"
1) negative 0.77
2) neutral 0.2107
3) positive 0.0193
##########################
Sentiment of the review: "This is not the best company in the world"
1) negative 0.9465
2) neutral 0.048
3) positive 0.0055
##########################
Sentiment of the review: "I kinda like this company"
1) positive 0.9588
2) neutral 0.0389
3) negative 0.0023
##########################
Sentiment of the review: "I am constantly learning something at this company"
1) positive 0.9436
2) neutral 0.0538
3) negative 0.0026
##########################
Sentiment of the review: "I'm glad I came here"
1) positive 0.9689
2) neutral 0.029
3) negative 0.0021
##########################
Sentiment of the review: "The company has been on the market for 10 years"
1) neutral 0.8452
2) negative 0.0948
3) positive 0.06
#####